In [1]:
import copy
import warnings
import pandas as pd
import numpy as np
from model import Company, StockLog, IndexLog, PerformanceLog
from model import start_engine
from datetime import date, datetime

warnings.filterwarnings("ignore")

In [2]:
engine = start_engine()

In [9]:
def get_table(
    stock,                      # list of stocks 
    start = '2009-07-01',       # start date in 'YYYY-MM-DD' format
    finish = None,              # finish date in 'YYYY-MM-DD' format
    engine = engine,
    change = False,             # daily value change    
    change_pct = False,         # daily percentage change    
    rank = False,               # daily rank
    # observations
    opening = False,
    high = False,
    low = False,
    closing = False,
    adjusted = False,
    volume = False,
    # fundamentals
    pe_ratio = False,
    dp_ratio = False,
    pb_ratio = False):
    
    start = datetime.strptime(start, '%Y-%m-%d').date()
    if finish:
        finish = datetime.strptime(finish, '%Y-%m-%d').date()
    else:
        finish = datetime.today()
    
    company = Company().query().get(stock)
    stock_log = StockLog().query().filter((StockLog.company == company) &
                                          (StockLog.date >= start) &
                                          (StockLog.date <= finish))
    df = pd.read_sql(stock_log.statement, engine)
    df.sort_values(by='date', inplace=True)
    df.set_index('date', inplace=True)
    table = pd.DataFrame(index=df.index)
    
    if opening:
        table['opening'] = df['opening']
    if high:
        table['high'] = df['high']
    if low:
        table['low'] = df['low']
    if closing:
        table['closing'] = df['closing']
    if adjusted:
        table['adjusted'] = df['adjusted']        
    if pe_ratio:
        table['pe_ratio'] = round(df['pe_ratio'] * 100, 2)
    if dp_ratio:
        table['dp_ratio'] = round(df['dp_ratio'], 2)
    if pb_ratio:
        table['pb_ratio'] = round(df['pb_ratio'], 2)
    if change:
        table['change'] = df['closing'].diff() 
    if change_pct:
        table['change_pct'] = round(df['change_pct'] * 100, 2)
    if rank:
        table['rank'] = df['rank']      
    
    table.sort_values(by='date', ascending=False, inplace=True)
    return table

In [10]:
stock = 'ABC'

In [11]:
get_table(stock, opening=1, high=1, low=1, closing=1, adjusted=1, pe_ratio=1, dp_ratio=1, pb_ratio=1, change=1, change_pct=1, rank=1).head()

,opening,high,low,closing,adjusted,pe_ratio,dp_ratio,pb_ratio,change,change_pct,rank
date,,,,,,,,,,,
2019-07-26,4.37,4.38,4.30,4.34,4.34,15.28,6.45,2.27,-0.05,-1.14,65
2019-07-25,4.34,4.40,4.32,4.39,4.39,15.46,6.38,2.30,0.06,1.39,22
2019-07-24,4.30,4.36,4.27,4.33,4.33,15.25,6.47,2.27,0.04,0.93,39
2019-07-23,4.44,4.48,4.26,4.29,4.29,15.11,6.53,2.25,-0.17,-3.81,78
2019-07-22,4.36,4.46,4.29,4.46,4.46,15.70,6.28,2.33,0.11,2.53,5


In [14]:
def get_rank(date, order='top', n=5, engine=engine):
    date = datetime.strptime(date, '%Y-%m-%d').date()
    stock_log = StockLog().query().filter(StockLog.date == date)
    df = pd.read_sql(stock_log.statement, engine)
    df.set_index('code', inplace=True)
    df['change_pct'] = round(df['change_pct'] * 100, 2)
    if order == 'top':
        df.sort_values(by='rank', ascending=False, inplace=True)
        df = df.nsmallest(n, 'rank')    
    else:
        df.sort_values(by='rank', inplace=True)
        df = df.nlargest(n, 'rank')
    df = df[['opening', 'high', 'low', 'closing', 'volume', 'change', 'change_pct']]
    return df

In [15]:
get_rank('2019-07-26', order='top', n=3)

,opening,high,low,closing,volume,change,change_pct
code,,,,,,,
RMD,18.22,19.14,18.07,18.99,2087007,1.00,5.56
RIO,96.57,98.42,96.39,98.26,3979357,1.97,2.05
JHX,19.50,19.86,19.38,19.78,1843555,0.36,1.85


In [16]:
get_rank('2019-07-26', order='bottom', n=3)

,opening,high,low,closing,volume,change,change_pct
code,,,,,,,
SHL,27.45,27.51,27.04,27.37,2399553,-0.78,-2.77
ALU,37.35,37.50,36.64,36.82,419806,-0.86,-2.28
CWY,2.51,2.52,2.46,2.46,3531027,-0.05,-1.99


In [17]:
#       ['code', 'revenue', 'expenses', 'ebitda', 'ebit', 'profit', 'npataa',
#        'assets', 'liabilities', 'operating_cash', 'investing_cash',
#        'financing_cash', 'shares', 'eps', 'dps', 'net_dividend',
#        'gross_dividend', 'net_yield', 'gross_yield', 'dividend_cover',
#        'total_equity', 'total_debt', 'invested_capital', 'working_capital',
#        'gross_investment', 'noplat', 'profit_margin', 'ebit_margin',
#        'ebita_margin', 'ebitda_margin', 'roe', 'roa', 'roic', 'noplat_margin',
#        'capital_turnover', 'inventory_turnover', 'asset_turnover',
#        'ppe_turnover', 'depreciation_ppe', 'depreciation_revenue',
#        'wkg_capital_revenue', 'wkg_capital_turnover', 'financial_leverage',
#        'gross_gearing', 'net_gearing', 'interest_cover', 'current_ratio',
#        'quick_ratio', 'gross_debt_cf', 'net_debt_cf', 'nta_ps', 'bv_ps',
#        'cash_ps', 'days_inventory', 'days_receivables', 'days_payables',
#        'cf_ps', 'sales_ps', 'share_price', 'market_cap', 'net_debt', 'ev',
#        'ev_ebit', 'ev_ebitda', 'market_cap_npat', 'market_cap_revenue',
#        'price_book_value', 'price_cash_flow', 'per', 'cash_asset',
#        'trade_asset', 'inventory_asset', 'equipment_asset', 'intangible_asset',
#        'goodwill_asset', 'nci_asset', 'other_asset', 'interest_income',
#        'non_interest_income', 'interest_expense', 'non_interest_expense',
#        'interest_margin', 'spread', 'non_interest_total', 'abnormals',
#        'pretax_profit', 'claims_ratio', 'expense_ratio', 'profitability',
#        'solvency']

def get_report(stock, tab='balance', start='2009-07-01', finish=None, engine=engine):
    tabs = {
        'balance': ['revenue', 'expenses', 'ebitda', 'ebit', 'profit', 'npataa', 'assets', 'liabilities'],
        'cash_flow': ['operating_cash', 'investing_cash', 'financing_cash'],
        'share_stats': ['shares', 'eps', 'dps', 'net_dividend', 'net_yield', 'dividend_cover'],
        'sundry': ['total_equity', 'total_debt', 'invested_capital', 'working_capital', 'noplat', 'profit_margin']
    }    
    start = datetime.strptime(start, '%Y-%m-%d').date()
    if finish:
        finish = datetime.strptime(finish, '%Y-%m-%d').date()
    else:
        finish = datetime.today()
    company = Company().query().get(stock)
    report_log = PerformanceLog().query().filter((PerformanceLog.company == company) &
                                                 (PerformanceLog.date >= start) &
                                                 (PerformanceLog.date <= finish))
    df = pd.read_sql(report_log.statement, engine)
    df.sort_values(by='date', ascending=False, inplace=True)
    df.set_index('date', inplace=True)
    df = df[tabs[tab]]
    return df

In [18]:
get_report('ABC', tab='share_stats')

,shares,eps,dps,net_dividend,net_yield,dividend_cover
date,,,,,,
2018-12-31,650610606.0,28.4,28.0,28.0,0.0656,1.450
2017-12-31,650272495.0,27.9,24.5,24.5,0.0376,1.327
2016-12-31,649654099.0,28.6,28.0,28.0,0.0516,1.430
2015-12-31,648885747.0,31.9,27.0,27.0,0.0568,1.668
2014-12-31,648267667.0,26.8,17.0,17.0,0.0476,1.633
2013-12-31,638456688.0,23.4,19.5,19.5,0.0531,1.418
2012-12-31,637387488.0,24.0,16.5,16.5,0.0529,1.454
2011-12-31,636277810.0,23.2,16.5,16.5,0.0571,1.406
2010-12-31,635132810.0,23.7,21.5,21.5,0.0652,1.436
